In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import json


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/netflix-movies-and-shows/imdb_movies_shows.csv


# Netflix Movies and Shows:
## Decoding Trends in Entertainment: A Deep Dive into Netflix's Content Library

In [2]:
df_netflix = pd.read_csv("/kaggle/input/netflix-movies-and-shows/imdb_movies_shows.csv").sort_values("release_year")

In [3]:
df_netflix.head()

,title,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes
0,Five Came Back: The Reference Films,SHOW,1945,TV-MA,48,['documentation'],['US'],1.0,NaN,NaN,NaN
24,Raya and Sakina,MOVIE,1953,NaN,105,"['drama', 'thriller', 'crime', 'history']",['EG'],NaN,tt0316472,6.8,231.0
28,The Blazing Sun,MOVIE,1954,NaN,100,"['romance', 'crime', 'drama']",['EG'],NaN,tt0044429,7.4,1219.0
15,White Christmas,MOVIE,1954,NaN,115,"['romance', 'comedy', 'music']",['US'],NaN,tt0047673,7.5,42373.0
22,Dark Waters,MOVIE,1956,NaN,120,"['drama', 'action', 'romance', 'thriller']",['EG'],NaN,tt0049761,6.7,590.0


In [4]:
px.histogram(df_netflix, x="type", color="type", labels={"type": "Type"})

In [5]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_netflix["release_year"].unique(),
        y=df_netflix.groupby("release_year").count()["imdb_score"],
        name="Movies + Shows",
        mode="lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=df_netflix.query("type == 'MOVIE'")["release_year"].unique(),
        y=df_netflix.groupby(["release_year", "type"])
        .count()
        .reset_index()
        .query("type == 'MOVIE'")["imdb_score"],
        name="Movies",
        mode="lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=df_netflix.query("type == 'SHOW'")["release_year"].unique(),
        y=df_netflix.groupby(["release_year", "type"])
        .count()
        .reset_index()
        .query("type == 'SHOW'")["imdb_score"],
        name="Shows",
        mode="lines+markers",
    )
)

fig.update_layout(
    title="Shows and movies released over the years",
    xaxis_title="Release Year",
    yaxis_title="Count",
    legend_title="Legend",
)
fig.show()

2019 is the year with the most **movies and shows** released, and it is the same
year with the most **movies** released. However, for **shows** it falls just
short of 2018 by 8 shows released.

# Separating Movies and Shows into two different dataframes

In [6]:
df_shows = df_netflix.loc[df_netflix["type"] == "SHOW"].drop(columns="type")
df_movies = df_netflix.loc[df_netflix["type"] == "MOVIE"].drop(columns="type")

## Distribution of the age rating of Shows and Movies in 2019

In [7]:
px.histogram(
    df_movies.loc[df_movies["release_year"] == 2019],
    x="age_certification",
    category_orders={"age_certification": ["G", "PG", "PG-13", "R", "NC-17"]},
    title="Movies in 2019 by age certification",
    labels={"age_certification": "age certification"},
    color="age_certification",
)

In [8]:
px.histogram(
    df_shows.loc[df_shows["release_year"] == 2019],
    x="age_certification",
    category_orders={
        "age_certification": ["TV-Y", "TV-Y7", "TV-G", "TV-PG", "TV-14", "TV-MA"]
    },
    color="age_certification",
    title="Shows in 2019 by age certification",
    labels={"age_certification": "age certification"},
)

In [9]:
fig = go.Figure()

x = df_movies.dropna(subset="imdb_score")["release_year"].unique()
df_movies_year_rating = df_movies.dropna(subset="imdb_score").groupby("release_year")[
    "imdb_score"
]

fig.add_trace(
    go.Scatter(
        x=x,
        y=df_movies_year_rating.max(),
        mode="lines+markers",
        name="Maximum IMDB rating",
    )
)

y_err = df_movies.groupby("release_year")["imdb_score"].sem().round(2)
fig.add_trace(
    go.Scatter(
        x=x,
        y=df_movies_year_rating.mean().round(2),
        mode="lines+markers",
        name="Average IMDB rating",
        error_y=dict(
            type="data",
            array=y_err,
            visible=True,
        ),
        hovertemplate="Year: %{x}"
        + "<br>Rating: %{y:.2f} +- %{customdata[0]:.2f}"
        + "<br>Count: %{customdata[1]}",
        customdata=np.stack((y_err, df_movies_year_rating.count().values), axis=-1),
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=df_movies_year_rating.min(),
        mode="lines+markers",
        name="Minimum IMDB rating",
    )
)

fig.update_yaxes(range=[0, 10])

fig.update_layout(
    title="Movie ratings across the years",
    xaxis_title="Release Year",
    yaxis_title="Rating",
    legend_title="Legend",
)

In [10]:
fig = go.Figure()

x = df_shows.dropna(subset="imdb_score")["release_year"].unique()

df_shows_year_rating = df_shows.dropna(subset="imdb_score").groupby("release_year")[
    "imdb_score"
]
fig.add_trace(
    go.Scatter(
        x=x,
        y=df_shows_year_rating.max(),
        mode="lines+markers",
        name="Maximum IMDB rating",
        hovertemplate="Year: %{x}" + "<br> Max: %{y}",
    )
)

y_err = df_shows_year_rating.sem().round(2)
fig.add_trace(
    go.Scatter(
        x=x,
        y=df_shows_year_rating.mean().round(2),
        mode="lines+markers",
        name="Average IMDB rating",
        error_y=dict(
            type="data",
            array=y_err,
            visible=True,
        ),
        hovertemplate="Year: %{x}"
        + "<br>Rating: %{y:.2f} +- %{customdata[0]:.2f}"
        + "<br>Count: %{customdata[1]}",
        customdata=np.stack((y_err, df_shows_year_rating.count().values), axis=-1),
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=df_shows_year_rating.min(),
        mode="lines+markers",
        name="Minimum IMDB rating",
        hovertemplate="Year: %{x}" + "<br> Min: %{y}",
    )
)

fig.update_yaxes(range=[0, 10])

fig.update_layout(
    title="Show ratings across the years",
    xaxis_title="Release Year",
    yaxis_title="Rating",
    legend_title="Legend",
)

It seems that from 2000 onwards we see a strong decrease in the 'worst'
rating given to productions across both movies and shows.